# Mastodon API operations

API operations defined in `mastodon_sim.mastodon_ops`.

In [1]:
%load_ext autoreload
%autoreload 2

## Check environment

Get environment variables with `.env` file (in project root directory)

In [2]:
from mastodon_sim.mastodon_ops import check_env

check_env()

2024-07-06T12:14:35.225689+0000 INFO Successfully loaded .env file.
2024-07-06T12:14:35.228336+0000 INFO API_BASE_URL: https://social-sandbox.com
2024-07-06T12:14:35.228808+0000 INFO EMAIL_PREFIX: austinmw89
2024-07-06T12:14:35.229175+0000 INFO MASTODON_CLIENT_ID: N*****************************************E
2024-07-06T12:14:35.229517+0000 INFO MASTODON_CLIENT_SECRET: b*****************************************s
2024-07-06T12:14:35.229840+0000 INFO USER0003_PASSWORD: f******************************6
2024-07-06T12:14:35.230197+0000 INFO USER0002_PASSWORD: 8******************************b
2024-07-06T12:14:35.230613+0000 INFO USER0004_PASSWORD: 7******************************e
2024-07-06T12:14:35.231191+0000 INFO USER0001_PASSWORD: 9******************************5
2024-07-06T12:14:35.231623+0000 INFO USER0005_PASSWORD: a******************************a


## Get Mastodon client

Create Mastodon client object

In [3]:
from mastodon_sim.mastodon_ops import get_client

mastodon = get_client()

## Login user

Get access token for `user0001`.

In [5]:
from mastodon_sim.mastodon_ops import login

access_token = login("user0001")
print(f"access_token: {access_token}")

access_token: pAvbKpkDFp3kOXZMlwRarowT2H7Ypj4LuyNo6UmX12g


## Follow user

Make `user0001` follow `user0002` and vice versa.

In [6]:
from mastodon_sim.mastodon_ops import follow

follow(login_user="user0001", follow_user="user0002")

follow(login_user="user0002", follow_user="user0001")

2024-07-06T12:16:12.894521+0000 INFO user0001 is now following user0002.
2024-07-06T12:16:13.430192+0000 INFO user0002 is now following user0001.


## Unfollow user

Make `user0002` unfollow `user0001`.

In [8]:
from mastodon_sim.mastodon_ops import unfollow

unfollow(login_user="user0002", unfollow_user="user0001")

2024-07-06T12:17:01.328796+0000 INFO user0002 has unfollowed user0001.


## Toot (post a message)

Make `user0001` toot a `message`.

Note that "toot" is analogous to "tweet".

In [9]:
from mastodon_sim.mastodon_ops import toot

toot(login_user="user0001", message="Hello World, from user0001!")

2024-07-06T12:17:29.865188+0000 INFO user0001 successfully posted the message.


## Get the public timeline

Get the entire pubic timeline in reverse chronological order, up to `limit`.

In [10]:
from mastodon_sim.mastodon_ops import get_public_timeline, print_timeline

timeline = get_public_timeline(limit=None)
print_timeline(timeline)

2024-07-06T12:17:36.317987+0000 INFO Retrieved 1 posts from the public timeline.
----------------------------------------
Post ID: 112739500017668416
Created At: 2024-07-06 12:17:29.792000+00:00
User:  (@user0001)
Content: <p>Hello World, from user0001!</p>
URL: https://social-sandbox.com/@user0001/112739500017668416
Favourites: 0, Reblogs: 0
----------------------------------------


## Get own timeline

Retrieve `user0002`'s timeline.

In [14]:
from mastodon_sim.mastodon_ops import get_own_timeline

timeline = get_own_timeline("user0002")
print_timeline(timeline)

2024-07-06T12:26:53.157778+0000 INFO user0002 retrieved 0 posts from their own timeline.


## Read a user's timeline (non-global)

From the perspective of `user0002`, read `user0001`'s timeline.

**Accessible Information**:
  - `user0002` can read `user0001`'s public and unlisted posts, as well as follower-only posts if `user0002` is a follower.
  - Details available include post ID, creation date, username, display name, content, URL, favourites count, and reblogs count.

**Restricted Information**:
  - `user0002` cannot access private posts or direct messages.

**Impact of Being Blocked**:
  - If `user0001` has blocked `user0002`, `user0002` cannot retrieve any posts from `user0001`'s timeline.
  - The function will return an empty list due to the block enforced by the Mastodon API.

In [15]:
from mastodon_sim.mastodon_ops import get_user_timeline

timeline = get_user_timeline(login_user="user0002", target_user="user0001", limit=None)
print_timeline(timeline)

2024-07-06T12:28:14.668574+0000 INFO user0002 retrieved 1 posts from user0001's timeline.
----------------------------------------
Post ID: 112739500017668416
Created At: 2024-07-06 12:17:29.792000+00:00
User:  (@user0001)
Content: <p>Hello World, from user0001!</p>
URL: https://social-sandbox.com/@user0001/112739500017668416
Favourites: 0, Reblogs: 0
----------------------------------------


## Block / unblock a user

Make `user0001` block or unblock `user0002`.

In [16]:
from mastodon_sim.mastodon_ops import block_user, unblock_user

# user001 blocks user002
block_user(login_user="user0001", target_user="user0002")

# user002 tries to read user001's timeline, but retrieves zero toots
timeline = get_user_timeline(login_user="user0002", target_user="user0001")

# user001 unblocks user002
unblock_user(login_user="user0001", target_user="user0002")

# user002 tries to read user001's timeline again, and retrieves the toots
timeline = get_user_timeline(login_user="user0002", target_user="user0001")

2024-07-06T12:28:34.566671+0000 INFO user0001 has blocked user0002.
2024-07-06T12:28:34.978172+0000 INFO user0002 retrieved 0 posts from user0001's timeline.
2024-07-06T12:28:35.436999+0000 INFO user0001 has unblocked user0002.
2024-07-06T12:28:35.933594+0000 INFO user0002 retrieved 1 posts from user0001's timeline.


## Like (favorite) a toot

Make `user0002` like `user0001`'s toot with ID `toot_id`.

In [17]:
from mastodon_sim.mastodon_ops import like_toot

timeline = get_user_timeline(login_user="user0002", target_user="user0001")

# Most recent toot is index 0 (reverse chronological order)
most_recent_toot_id = timeline[0]["id"]

like_toot(login_user="user0002", target_user="user0001", toot_id=most_recent_toot_id)

2024-07-06T12:29:13.653197+0000 INFO user0002 retrieved 1 posts from user0001's timeline.
2024-07-06T12:29:14.219481+0000 INFO user0002 liked post 112739500017668416 from user0001.


## Boost (reblog) a toot

Make `user0002` boost `user0001`'s toot with ID `toot_id`.

In [18]:
from mastodon_sim.mastodon_ops import boost_toot

timeline = get_user_timeline(login_user="user0002", target_user="user0001")
most_recent_toot_id = timeline[0]["id"]

boost_toot(login_user="user0002", target_user="user0001", toot_id=most_recent_toot_id)

2024-07-06T12:29:38.307035+0000 INFO user0002 retrieved 1 posts from user0001's timeline.
2024-07-06T12:29:38.844742+0000 INFO user0002 boosted post 112739500017668416 from user0001.


## Update name and bio

Update `user0001`'s display name and bio.

In [19]:
from mastodon_sim.mastodon_ops import update_bio

display_name = "Bob"
bio = "Hello, I am user0001!"

update_bio(login_user="user0001", display_name=display_name, bio=bio)

2024-07-06T12:30:23.028276+0000 INFO user0001 successfully updated the display name and bio.


## Read name and bio

Have one user read another user's bio. Defaults to "Not provided" if the user has not set either.

In [20]:
from mastodon_sim.mastodon_ops import read_bio

display_name, bio = read_bio(login_user="user0002", target_user="user0001")

display_name, bio = read_bio(login_user="user0001", target_user="user0002")

2024-07-06T12:30:38.308841+0000 INFO User: user0001
Display Name: Bob
Bio: <p>Hello, I am user0001!</p>
2024-07-06T12:30:38.781627+0000 INFO User: user0002
Display Name: Not provided
Bio: Not provided


And that's it so far!

## [TODO] Operations to add:

- notifications
- notifications_clear
- status_post
- delete_status
- favourites
- bookmarks
- make_poll
- media_post
- account_mute
- account_unmute
- timeline_hashtag
- conversations
- trending_tags
- admin_account_moderate

https://mastodonpy.readthedocs.io/en/stable/index.html